# Plan for the Week 

- Talking about Recommender Systems
- What kind of data would you need for your project? 
- Monday: Basic Recommender Code 
- Wednesday: Including Spotify Features


# Monday

The code below generates fake ratings for each user for a bunch of songs: 


In [2]:
import pandas as pd
import random

# Define users and songs
users = ['Alex', 'Stephanie', 'Hannah', 'Kelvin', 'Khoi', 'Leo', 'Ally', 'Jordan', 'Sid', 'Jacob']
songs = [
    'Respect', 'In The Mood', 'White Christmas', 'Johnny B. Goode', 'Over The Rainbow',
    'Thriller', 'Imagine', 'Billie Jean', 'Hotel California', 'Stairway to Heaven', 'Shake it Off', 
    'As it Was', 'Truth Hurts', 'All I Want for Christmas', 'Fade to Black', 'Enter Sandman', 'Smells Like Teen Spirit',
    'Paranoid Android', 'Sgt. Pepper', 'Help', 'Good Vibrations', 'Happy', '2 Legit 2 Quit', 'Happy Birthday', 'Happy', 
    'The Theme from the 90s TV Show E.R', 'Elephants Playing Gongs', 'Raindrops Keep Falling on My Head', 'Beethoven 9',
    'Dolphins Singing', 'The NU Fight Song', 'Bach?', 'Do you really want to hurt me?', 'Schools Out', 'Jesses Girl',
    'Dan Shanahan Yelling', 'Dan Shanahan Rocking on the Guitar', 'Sounds of a Campfire'

]

# Generate random ratings
data = []
for i in range(2000):
    user = random.choice(users)
    song = random.choice(songs)
    rating = random.randint(1, 5)
    data.append({'user_id': user, 'track_title': song, 'ratings': rating})

# Create DataFrame and save to CSV
df = pd.DataFrame(data)
# if you want to save it as a csv:
# df.to_csv('spotify_data.csv', index=False)
print(df.head(100))

      user_id                         track_title  ratings
0        Khoi                             Imagine        2
1         Leo                Sounds of a Campfire        5
2        Khoi                Sounds of a Campfire        4
3         Leo                     Johnny B. Goode        5
4         Sid                      2 Legit 2 Quit        5
..        ...                                 ...      ...
95  Stephanie                   The NU Fight Song        1
96       Khoi                       Fade to Black        5
97       Khoi                     Good Vibrations        5
98       Khoi  Dan Shanahan Rocking on the Guitar        4
99       Khoi                         Schools Out        4

[100 rows x 3 columns]


It ends up looking something like this:

```{txt}
      user_id       track_title  ratings
0         Leo       Schools Out        2
1      Jordan     Enter Sandman        4
2       Jacob  Over The Rainbow        1
3      Jordan  Paranoid Android        4
4      Hannah     Fade to Black        1
..        ...               ...      ...
95        Leo       Jesses Girl        4
96      Jacob             Happy        4
97  Stephanie     Fade to Black        4
98     Kelvin       Truth Hurts        2
99        Leo              Help        1
```

### Beginning Recommender Code with Surprise

First, we will begin by loading all of the libraries. Here, I'm going to be using pandas, and a number of modules from the Surprise toolkit. If you don't already have it installed, typing `pip intall surprise` into your Python environment will install it for you.

Once the libraries are loaded, you can import the dataframe `df` that we created above, and load it into surprise's `Reader` object. Then, we can do an 80/20 training and testing split quite easily with the built-in train_test_split function.

In [3]:
import pandas as pd
from surprise import Dataset, Reader, SVD, KNNBasic, KNNWithMeans
from surprise.model_selection import train_test_split


# Create a Surprise Reader object
reader = Reader(rating_scale=(1, 5))

# Load the data into the Surprise Dataset format
data = Dataset.load_from_df(df[['user_id', 'track_title', 'ratings']], reader)

# Split the data into training and testing sets
trainset, testset = train_test_split(data, test_size=0.20)

Now it's time to, in effect, fill in the blank spots in our matrix. Surprise has a number of algorithms built in. Singular Value Decomposition (SVD) is probably the most commonly-used one, but there are many KNNs bundled, as well. Here I've used the KNNBasic. I then use that algorithm to train on the training dataset, and then predict with the testing set.

In [4]:
# Use SVD algorithm
# algo = SVD()
algo = KNNBasic()
# algo = KNNWithMeans()

# Train the algorithm on the trainset
algo.fit(trainset)

# Make predictions on the testset
predictions = algo.test(testset)

Computing the msd similarity matrix...
Done computing similarity matrix.


I could also look at a content based system with a different form of similarity, here using a geometric approach, if I wanted:

In [5]:
sim_options = {
    "name": "cosine",
    "user_based": False,  # compute  similarities between items, not users!!!
}
algo = KNNBasic(sim_options=sim_options)
# algo = KNNWithMeans()


Here are some functions for getting the top recommendations for each user. Note that I loop over the user id, item id, and rating, as well as the estimate. I then grab the top 10 from these and sort the data.

In [6]:
# Function to get top N recommendations for a user
def get_top_n_recommendations(predictions, n=10):
    top_n = {}
    for user_id, item_id, true_rating, est, i in predictions:
        if user_id not in top_n:
            top_n[user_id] = []
        top_n[user_id].append((item_id, est))
    
    # Sort the predictions for each user and retrieve the top ones
    for user_id, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[user_id] = user_ratings[:n]
    
    return top_n

# Get top 5 recommendations for each user
top_n = get_top_n_recommendations(predictions, n=5)

This is just a printing function.

In [7]:
# Print the top 5 recommendations
for i, (user, recommendations) in enumerate(list(top_n.items())):
    print(f"\nTop track recommendations for user {user}:")
    for j, (track_id, estimated_rating) in enumerate(recommendations, 10):
        print(f"Track Title: {track_id}, Estimated Rating: {estimated_rating:.2f}")


Top track recommendations for user Jordan:
Track Title: Dan Shanahan Rocking on the Guitar, Estimated Rating: 3.51
Track Title: Happy, Estimated Rating: 3.42
Track Title: Happy, Estimated Rating: 3.42
Track Title: Happy, Estimated Rating: 3.42
Track Title: Happy, Estimated Rating: 3.42

Top track recommendations for user Stephanie:
Track Title: Dolphins Singing, Estimated Rating: 3.65
Track Title: Dolphins Singing, Estimated Rating: 3.65
Track Title: Dolphins Singing, Estimated Rating: 3.65
Track Title: Schools Out, Estimated Rating: 3.60
Track Title: Schools Out, Estimated Rating: 3.60

Top track recommendations for user Alex:
Track Title: Stairway to Heaven, Estimated Rating: 3.77
Track Title: Elephants Playing Gongs, Estimated Rating: 3.52
Track Title: Happy Birthday, Estimated Rating: 3.46
Track Title: Happy Birthday, Estimated Rating: 3.46
Track Title: Happy Birthday, Estimated Rating: 3.46

Top track recommendations for user Kelvin:
Track Title: Schools Out, Estimated Rating: 3.

And this gets the following results:

```{txt}
Computing the msd similarity matrix...
Done computing similarity matrix.

Top track recommendations for user Khoi:
Track Title: Schools Out, Estimated Rating: 3.38
Track Title: Help, Estimated Rating: 3.29
Track Title: Beethoven 9, Estimated Rating: 3.18
Track Title: Over The Rainbow, Estimated Rating: 3.16
Track Title: Stairway to Heaven, Estimated Rating: 3.14

Top track recommendations for user Hannah:
Track Title: Good Vibrations, Estimated Rating: 3.39
Track Title: Good Vibrations, Estimated Rating: 3.39
Track Title: Good Vibrations, Estimated Rating: 3.39
Track Title: Imagine, Estimated Rating: 3.32
Track Title: Imagine, Estimated Rating: 3.32

etc.
```

The standard way for evaluating the accuracy of the algorithm is the RMSE (root mean square error). Again, Surprise has a built-in method for this:

In [8]:
#rmse = accuracy.rmse(predictions)

And we end up with:

```{txt}
RMSE: 1.4986
```

This isn't great. I think for movies, I've seen people shoot for .8; for other things it's even better than that (lower is better).

### Visualizing the Similarity

We can look at the similarity of items with a bit of help from Seaborn:

In [9]:
# import matplotlib.pyplot as plt
# import seaborn as sns
# # Extract the similarity matrix
# sim_matrix = algo.sim
# 
# # Convert to a pandas DataFrame
# df_sim = pd.DataFrame(sim_matrix)

# Create a heatmap
# plt.figure(figsize=(12, 10))
# sns.heatmap(df_sim, cmap='YlGnBu')
# plt.title('Item Similarity Matrix Heatmap')
# plt.show()

This will give us this:
![Item Similarity Matrix](../images/item-similarity-matrix.png)


## The Cold Start Problem

This was all random data. How will you get data for your project? What will it look like for your model to be accurate?


## Two Spotify Discover Playlists

What if I take my own discover playlist, and find someone else's and see what it might recommend?

We can start by getting data from playlists. Put in your own Spotify data here. I was having some cache issues with my client id, so I'm using the `MemoryCacheHandler` function from Spotipy.

In [10]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from surprise import Dataset, Reader, KNNBasic
import pandas as pd
from spotipy.cache_handler import MemoryCacheHandler


client_credentials_manager = SpotifyClientCredentials(
    client_id = 'YOUR OWN CLIENT ID',
    client_secret = 'YOUR OWN CLIENT SECRET',
    cache_handler=MemoryCacheHandler()
)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

Here's a function that grabs the audio data from Spotify, using spotipy and pandas:

In [11]:
# Function to get playlist tracks and audio features
def get_playlist_data(playlist_id):
    results = sp.playlist_tracks(playlist_id)
    tracks = results['items']
    
    data = []
    for track in tracks:
        track_id = track['track']['id']
        audio_features = sp.audio_features(track_id)[0]
        
        data.append({
            'track_id': track_id,
            'acousticness': audio_features['acousticness'],
            'danceability': audio_features['danceability'],
            'energy': audio_features['energy'],
            'instrumentalness': audio_features['instrumentalness'],
            'liveness': audio_features['liveness']
        })
    
    return pd.DataFrame(data)


And we can add all of the Spotify URIs for the playlist:

In [12]:
# Get data for both playlists
playlist1_id = '37i9dQZEVXcIDdjTZ1FKnJ'
playlist2_id = '2VYB4MjPYJx3ZOkrjyUvsx'

df1 = get_playlist_data(playlist1_id)
df2 = get_playlist_data(playlist2_id)

SpotifyOauthError: error: invalid_client, error_description: Invalid client

Now we will read it into Surprise with the Reader function:


In [ ]:
# Assign user IDs (1 for playlist1, 2 for playlist2)
df1['user_id'] = 'Dannah Shanalee'
df2['user_id'] = 'Hannah Ashlee'

# Combine dataframes
combined_df = pd.concat([df1, df2], ignore_index=True)

# Prepare data for Surprise
reader = Reader(rating_scale=(0, 1))
data = Dataset.load_from_df(combined_df[['user_id', 'track_id', 'acousticness']], reader)

Here we have a function that gets the track information (name, artist, and id) to print it later (it was only printing URIs), and then recommend 5 tracks from the other playlist:

In [ ]:
def get_track_info(track_id):
    try:
        track_info = sp.track(track_id)
        return {
            'name': track_info['name'],
            'artist': track_info['artists'][0]['name'],
            'id': track_id
        }
    except:
        return {'name': 'Unknown', 'artist': 'Unknown', 'id': track_id}

def get_recommendations(track_id, model, n=5):
    inner_id = model.trainset.to_inner_iid(track_id)
    neighbors = model.get_neighbors(inner_id, k=n)
    raw_neighbors = [model.trainset.to_raw_iid(inner_id) for inner_id in neighbors]
    return [get_track_info(track_id) for track_id in raw_neighbors]

# Example: Get recommendations for a track
sample_track = combined_df['track_id'].iloc[0]
sample_track_info = get_track_info(sample_track)
recommendations = get_recommendations(sample_track, algo)

print(f"Recommendations for '{sample_track_info['name']}' by {sample_track_info['artist']}:")
for rec in recommendations:
    print(f"- '{rec['name']}' by {rec['artist']} (ID: {rec['id']})")

And we get five new recommendations:

```{txt}

Recommendations for 'Lily Pad on Your Doorstep' by Don't Stop or We'll Die:
- 'Amazing Glow' by Pernice Brothers (ID: 6ViAxolMqWD1d6JrqgPZKc)
- 'Rang Tang Ring Toon' by Mountain Man (ID: 2HZRjBrPeo3HwyZVUZxK62)
- 'Amsterdam' by Guster (ID: 3fv9cBtpMOaFaIAO4uVRBV)
- 'Bernadette' by Elle Cordova (ID: 4NFhb4AOPBulDFgxyoXaLH)
- 'Tire Swing' by Kimya Dawson (ID: 0vbhRDi46TDNHkhKbZa81B)
```

After working through this example, in class, we realized that it wasn't really doing what we thought it was doing. We thought it was picking the top tracks from the other playlist, but in fact it was taking a *single track* and picking five similar tracks from *that same playlist*. 

So for Wednesday we want to do a few things:

1. Look at two playlists, and **use the entire set of songs** to predict a single new song from playlist 1 for the user of playlist 2.
2. We want to compare a few different models and see what this looks like, comparing the accuracy as needed.

# Wednesday

So let's start from scratch.

## Importing the Libraries and the Data

First, I'm going to import the spotipy, pandas, and surprise. I'm also going to import specific authentication and cache things with spotipy, and some models with Surprise.

In [ ]:
import spotipy
import os
from surprise import Dataset, Reader, KNNBasic, SVD
import pandas as pd
from spotipy.cache_handler import MemoryCacheHandler
from surprise.model_selection import cross_validate
from spotipy.oauth2 import SpotifyClientCredentials

I've stored my id stuff locally so you can't add things to my playlists...

In [ ]:
import os
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# Access environment variables
client_id = os.getenv('SPOTIFY_CLIENT_ID')
client_secret = os.getenv('SPOTIFY_CLIENT_SECRET')

# Set up Spotify client credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id, client_secret=client_secret))


In [ ]:
import os

print(os.getenv('SPOTIFY_CLIENT_ID'))
print(os.getenv('SPOTIFY_CLIENT_SECRET'))

Now that those are loaded, let's grab the data from two playlists from Spotify. 

We will start with a function that takes the playlist uri as an argument, grabs the audio features from the tracks:

In [ ]:
# Function to get playlist tracks and audio features
def get_playlist_data(playlist_id):
    results = sp.playlist_tracks(playlist_id)
    tracks = results['items']
    
    data = []
    for track in tracks:
        track_id = track['track']['id']
        audio_features = sp.audio_features(track_id)[0]
        
        data.append({
            'track_id': track_id,
            'acousticness': audio_features['acousticness'],
            'danceability': audio_features['danceability'],
            'energy': audio_features['energy'],
            'instrumentalness': audio_features['instrumentalness'],
            'liveness': audio_features['liveness'],
            'mode': audio_features['mode'],
            'tempo': audio_features['tempo'],


        })
    
    return pd.DataFrame(data)

We can use this function and grab the information from the playlist with the following:

In [ ]:
# Get data for both playlists
playlist1_id = '37i9dQZEVXcIDdjTZ1FKnJ'
playlist2_id = '2VYB4MjPYJx3ZOkrjyUvsx'

df1 = get_playlist_data(playlist1_id)
df2 = get_playlist_data(playlist2_id)

Let's see what kind of data we have from this:

In [ ]:
df1

Note how it gets the track URI, and then all of the variables we asked for. All of these are continuous, with the exception of mode, which is a binary (0 or 1).

We can add our user data here:

In [ ]:
# Assign user IDs (1 for playlist1, 2 for playlist2)
df1['user_id'] = 'Dannah Shanalee'
df2['user_id'] = 'Hannah Ashlee'

Now we can see the data has specific user names (here only the first five rows).

In [ ]:
df1.head()

We can combine the two dataframes into one here:

In [ ]:
combined_df = pd.concat([df1, df2], ignore_index=True)
combined_df['user_id'].value_counts()

Here we see (using the `value_counts` function in `pandas`), that there are 100 songs in Hannah Ashlee's list, and 30 in mine. Not ideal, but we will try to see where it takes us.

Now we will load the data into the surprise toolkkit, using the `Reader` function. The rating_scale option gives it a range of ratings. Here it's just 0 or 1, as it's if it's present in a playlist or not, but it could also be from 1 to 5 (if it's reviews) or something like that. 

In [ ]:
reader = Reader(rating_scale=(0, 1))

The `load_from_df` function in `surprise` is meant to accept three things: the user id, the item id, and a rating. Here, we don't really have a rating, which again isn't ideal. So here I'm just adding in a third thing (acousticness).

In [ ]:
data = Dataset.load_from_df(combined_df[['user_id', 'track_id', 'acousticness']], reader)

## Building an Item-Item Recommender System

Here I'm going to look at the `sim_options` argument for surprise, which is short for "similarity options". 

There are a few options to try for the name argument:

1. `cosine` give us [cosine similarity](https://en.wikipedia.org/wiki/Cosine_similarity)
2. `msd` gives use the [mean squared difference similarity](https://en.wikipedia.org/wiki/Mean_squared_error)
3. `pearson` gives us the [pearson correlation coefficient](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient)
4. `pearson_baseline` gives us the pearson correlation coefficient with a baseline estimate for ratings, which is meant to account for the overall average rating, and any user or item bias. 

For now, we can just use `pearson`, but it's worth comparing:

In [ ]:
## I'm going to give options for the model here, including specific model, and whether it's item based or user based.
sim_options = {
    'name': 'msd',
    'user_based': False  # Item-based similarity
}

Here is my code for implementing a K-nearest neighbor:

In [ ]:
dans_algorithm = KNNBasic(sim_options=sim_options)
trainset = data.build_full_trainset()
dans_algorithm.fit(trainset)
sim_matrix = dans_algorithm.compute_similarities()

This gives us a similarity matrix between items. Here I'll convert it to a pandas dataframe:

In [ ]:
sim_matrix
# Convert to a pandas DataFrame
df_sim = pd.DataFrame(sim_matrix)
df_sim

This is telling me pairwise similarities using msd between the songs in the columns and the songs in the rows.

Now that we have similarities, and compare between lists.


## Using Data from One List to Recommend to Another

Let's start with a function to grab the track info based on the URI. This isn't necessary, but if we want to be able to interpret anything past the URI, we should do this. 

NOTE: I kept getting errors because of Unknown IDs, it would choke the Spotify API up here. the `try` and `except` are meant to get around that.

In [ ]:
def get_track_info(track_id):
    try:
        track_info = sp.track(track_id)
        return {
            'name': track_info['name'],
            'artist': track_info['artists'][0]['name'],
            'id': track_id
        }
    except:
        return {'name': 'Unknown', 'artist': 'Unknown', 'id': track_id}

Now we can take those original track URIs and write a function to get the recommendations. Here, we get the "inner id", which is basically an internal representation used by `surprise`. Then we get the neighbors. 

A couple of other things:

- k is referring to the amount of neighbors; here I've set it to 10.

In [ ]:
def get_recommendations(track_id, model, n=10):
    inner_id = model.trainset.to_inner_iid(track_id)
    neighbors = model.get_neighbors(inner_id, k=n)
    raw_neighbors = [model.trainset.to_raw_iid(inner_id) for inner_id in neighbors]
    return [get_track_info(track_id) for track_id in raw_neighbors]

Here is code to get the recommendations *for myself* based on this other playlist:

In [ ]:
# Get recommendations for the user from playlist 2
user_id = 'Dannah Shanalee'
playlist2_tracks = df2['track_id'].tolist()
recommendations = []
for track_id in playlist2_tracks:
    recs = get_recommendations(track_id, dans_algorithm, n=10)
    recommendations.extend(recs)

Here are the top 10 recommendations in a pandas dataframe:

In [ ]:
pd.DataFrame(recommendations).head(10)

Now it might be worth using the `cross_validate` function to see how well this did.

In the code below, I use a 5-fold cross validation. Just as a reminder, this does the following steps:

1. Partitions the data into 5 "folds"
2. It then trains the data on 4 folds, and tests it on the other.

Then, I look at both "RMSE" (Root Mean Square Error) and "MAE" (Mean Absolute Error). 

In [ ]:
results = cross_validate(dans_algorithm, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

And as a pandas dataframe:

In [1]:
pd_results = pd.DataFrame(results)
pd_results

NameError: name 'pd' is not defined

I had to look up what "good" RMSE numbers might be for this, and it seems that with a binary rating (as we have here) or lower is considered "good", so I think we've done a decent job here.

# Some Future Steps

1. Here we don't have ratings, which would have actually made this much easier. How might you get ratings?
2. Here we are just using audio data, but metadata would really help. We could look at things like genre or artist between playlists, and pre-process the data with a one-hot encoding or something like that.

This unworkable code might be one way to do it?:

In [ ]:
# import pandas as pd
# from sklearn.preprocessing import MultiLabelBinarizer
# 
# # Example DataFrame
# data = {
#     'user_id': [1, 1, 2, 2, 3, 3],
#     'track_id': [101, 102, 101, 103, 102, 104],
#     'genres': [['rock', 'pop'], ['pop'], ['rock'], ['jazz'], ['pop', 'jazz'], ['rock', 'jazz']]
# }
# df = pd.DataFrame(data)
# 
# # One-hot encode the genres
# mlb = MultiLabelBinarizer()
# genre_encoded = mlb.fit_transform(df['genres'])
# genre_df = pd.DataFrame(genre_encoded, columns=mlb.classes_)
# 
# # Combine with user_id and track_id
# combined_df = pd.concat([df[['user_id', 'track_id']], genre_df], axis=1)
# 
# # Display the combined DataFrame
# print(combined_df)